In [170]:
!pip install opencv-python-headless
!pip install opencv-python
!pip install scipy
!pip3 install pandas

In [1]:
import scipy.io
import cv2
import pandas as pd
import os
import random 
import numpy as np
import json
import math

In [71]:
def parseVideoName(video_name):
    video_name = video_name.replace('.avi', '')
    first_part,second_part = video_name.split(',')
    video_type, num_targets = first_part.rsplit('_', 1)
    videos_info = second_part.split('_')
    
    parsed_data = {
        'video_name': video_name,
        'target_name': second_part,
        'video_type': video_type,
        'num_targets': int(num_targets),
        'first_video': int(videos_info[0]),
        'second_video': int(videos_info[1]),
        'first_start_frame': int(videos_info[2]),
        'second_start_frame': int(videos_info[3]),
        'first_rotation': float(videos_info[4]),
        'second_rotation': float(videos_info[5])
    }
    
    return parsed_data
    
def generateRandomColor():
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

def processTrialsVideos(file_path):
    df = pd.read_csv(file_path)
    parsed_rows = []
    
    for index, row in df.iterrows():
        video_name = row['videoName']
        parsed_data = parseVideoName(video_name)
        parsed_data['firstTargets'] = row['firstTargets']
        parsed_data['secondTargets'] = row['secondTargets']
        parsed_rows.append(parsed_data)
        # parsed_rows.append(1)
    return pd.DataFrame(parsed_rows)
    
def loadMatFile():
    root_folder = 'GazeMat'
    test_subs = ['sub11']

    data = {}
    
    for sub_dir in os.listdir(root_folder):
        if sub_dir in test_subs:
            mat_file_path = os.path.join(root_folder, sub_dir, '40.mat')
            if os.path.exists(mat_file_path):
                mat_data = scipy.io.loadmat(mat_file_path)
                data[sub_dir] = mat_data["extractedData"]
            else:
                print(f"{mat_file_path} does not exist.")
    print("Data loaded successfully.")
    return data


def loadShapeFishPoints(videoName,numberFrame):
    json_filename = f'img{numberFrame:03d}.json'
    json_path = os.path.join('./jsons', str(videoName), json_filename)

    if not os.path.exists(json_path):
        raise FileNotFoundError(f"JSON file not found: {json_path}")

    with open(json_path, 'r') as json_file:
        data = json.load(json_file)
    
    return data.get('shapes', [])

def getTargetsFromJson(shapes,targets):
    if not isinstance(targets, (list, tuple)):
        targets = [targets]
    
    target_digits = set()
    for target in targets:
        if isinstance(target, int):
            target_digits.update(str(target))
        else:
            raise ValueError("Targets should be integers or lists/tuples of integers")

    filtered_shapes = [shape for shape in shapes if shape['label'] in target_digits]
    
    return filtered_shapes

def getTotalShape(shapeInfo):
    firstFrame = shapeInfo['firstStart']
    secondFrame = shapeInfo['secondStart']
    firstTargets = shapeInfo['firstTargets']
    secondTargets = shapeInfo['secondTargets']
    shapes_video1 = loadShapeFishPoints(shapeInfo['first'], firstFrame)
    shapes_video2 = loadShapeFishPoints(shapeInfo['second'], secondFrame)
    merged_shapes =  getTargetsFromJson(shapes_video1,firstTargets) + getTargetsFromJson(shapes_video2,secondTargets)
    # print(merged_shapes)
    return merged_shapes

def generateSubjectName(subIndex):
    number_part = subIndex[3:]
    modified_number = number_part[:-1] if len(number_part) > 1 else '0'
    return f'sub{modified_number}'

def getVideoInfo(video):
    fps = video.get(cv2.CAP_PROP_FPS)
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    # num_frames = int(mainVideo.get(cv2.CAP_PROP_FRAME_COUNT))
    return fps, width, height 

def createOutputVideo(video,name):
    output_gaze_videos = 'gazeOutputVideo'
    os.makedirs(output_gaze_videos, exist_ok=True)
    fps, width, height = getVideoInfo(video)
    
    output_gaze_videos_path = os.path.join(output_gaze_videos, name + '.avi')
    output_video = cv2.VideoWriter(output_gaze_videos_path, cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height))
    return output_video

def getSubjectColor(subject):
    if subject not in subject_colors:
        subject_colors[subject] = generateRandomColor()
    return subject_colors[subIndex]

def mapPointsOnFrame(frame):
    return round(frame * (7798 / 110))

def openVideo(video):
    trialVideo = cv2.VideoCapture(os.path.join('./videos', video.video_type, video.target_name + '.avi'))
    
    if not trialVideo.isOpened():
        raise ValueError(f"Unable to open video file: {os.path.join('./videos', video.video_type, video.target_name + '.avi')}")
    return trialVideo

def generateXPoints(x):
    scale_x = 660 / 1920
    return int(x * scale_x)

def generateYPoints(y):
    scale_y = 660 / 1080
    return int(y * scale_y)

In [72]:
subjects = loadMatFile()
subject_colors = {}
subjects['sub11'].shape

Data loaded successfully.


(40, 2)

In [73]:
### to Check data 

for index in subjects:
    x_points = subjects[index][0]  # First column for x points
    y_points = subjects[index][1]
# print(check_all_elements_type(subjects['sub11'][1][0]))
trialNumber  = 0

# Pay attentoion: it is from 0 to 39
xc = subjects['sub11'][trialNumber][0] # (7798, 1) => [1017.29998779]
yc = subjects['sub11'][trialNumber][1]

xc = subjects['sub11'][1][0][3445][0]
yc = subjects['sub11'][1][1]

print(subjects['sub11'][39][0].shape)

(7797, 1)


In [74]:
csvSample = processTrialsVideos('./sub11.csv')
# We are checking the sub11 gaze mat file and its trials
# so we should see sub11 csv file
csvSample

,video_name,target_name,video_type,num_targets,first_video,second_video,first_start_frame,second_start_frame,first_rotation,second_rotation,firstTargets,secondTargets
0,"NO_2,1_10_1_1_45_315",1_10_1_1_45_315,NO,2,1,10,1,1,45.0,315.0,12,0
1,"NO_2,1_10_119_344_45_90",1_10_119_344_45_90,NO,2,1,10,119,344,45.0,90.0,3,4
2,"OB_4,1_10_1_344_45_315",1_10_1_344_45_315,OB,4,1,10,1,344,45.0,315.0,3412,0
3,"OB_2,10_10_114_229_0_135",10_10_114_229_0_135,OB,2,10,10,114,229,0.0,135.0,14,0
4,"NO_4,10_10_229_344_0_315",10_10_229_344_0_315,NO,4,10,10,229,344,0.0,315.0,43,41
5,"OB_2,1_10_1_1_45_0",1_10_1_1_45_0,OB,2,1,10,1,1,45.0,0.0,0,14
6,"OC_4,1_10_119_344_0_247.5",1_10_119_344_0_247.5,OC,4,1,10,119,344,0.0,247.5,13,23
7,"OB_OC_4,1_10_119_114_45_225",1_10_119_114_45_225,OB_OC,4,1,10,119,114,45.0,225.0,342,1
8,"OC_4,1_10_119_344_45_90",1_10_119_344_45_90,OC,4,1,10,119,344,45.0,90.0,31,41
9,"OB_4,1_10_119_229_45_225",1_10_119_229_45_225,OB,4,1,10,119,229,45.0,225.0,342,4


In [80]:
videoList = []
trialNumber  = 1

for index, video in csvSample.iterrows():
    try:
        
        open_video = openVideo(video)
        output_video = createOutputVideo(open_video,video.video_name)
        
        frame_idx = 0

        while True:
            ret, frame = open_video.read()
            if not ret:
                break  

                     
            for subIndex in subjects:
                x_points = subjects[subIndex][trialNumber][0]  # (7798, 1)
                y_points = subjects[subIndex][trialNumber][1]  # (7798, 1)

                # We select points from range 0 to 7798 (the size of points)
                point_add = mapPointsOnFrame(frame_idx)
                
                print(point_add)
                if 0 <= point_add < len(x_points):
                    x = x_points[point_add][0] # the index 0 is because of the structure, they have just one item 
                    y = y_points[point_add][0] 

                    if math.isnan(x):
                        if previous_x is not None:
                            x = previous_x
                        else:
                            print(f"NaN detected for x at frame {frame_idx}, but no previous value to replace. Skipping.")
                            continue

                    if math.isnan(y):
                        if previous_y is not None:
                            y = previous_y
                        else:
                            print(f"NaN detected for y at frame {frame_idx}, but no previous value to replace. Skipping.")
                            continue
            
                    # Store the current valid value as previous
                    previous_x = x
                    previous_y = y
                    
                    new_x = generateXPoints(x)
                    new_y = generateYPoints(y)
                    print(x,'(x) -> ', new_x, y, '(y) -> ', new_y)
                    color = getSubjectColor(subIndex)

                    cv2.circle(frame, (new_x, new_y), 5, color, 3) 

                    cv2.putText(frame, generateSubjectName(subIndex), (new_x + 5, new_y - 5),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)

            shapeInfo = {
                'first': video.first_video,
                'second': video.second_video,
                'firstTargets': video.firstTargets,
                'secondTargets': video.secondTargets,
                'firstStart': video.first_start_frame,
                'secondStart': video.second_start_frame
            }
            shapes = getTotalShape(shapeInfo)  
            # if(index == 0):
            #     # print(shapes)
            for shape in shapes:
                points = shape['points']
                calibrated_points = [(x, y) for x, y in points]
                cv2.polylines(frame, [np.array(calibrated_points)], isClosed=True, color=(0, 0, 225), thickness=2)
                cv2.fillPoly(frame, [np.array(calibrated_points)], color=(0, 0, 225))

            resized_frame = cv2.resize(frame, (660, 660))
            output_video.write(resized_frame)
            
            frame_idx += 1
        
        trialNumber += 1
        
    except Exception as e:
        print(f"An error occurred while processing video at index {index}: {e}")

    finally:
        # Release everything
        if 'open_video' in locals() and open_video.isOpened():
            open_video.release()
        if 'output_video' in locals():
            output_video.release()
        cv2.destroyAllWindows()
    

0
909.4000244140625 (x) ->  312 296.5 (y) ->  181
71
826.9000244140625 (x) ->  284 383.20001220703125 (y) ->  234
142
827.0 (x) ->  284 386.79998779296875 (y) ->  236
213
821.2999877929688 (x) ->  282 383.0 (y) ->  234
284
823.5 (x) ->  283 383.79998779296875 (y) ->  234
354
822.7999877929688 (x) ->  282 382.5 (y) ->  233
425
822.7999877929688 (x) ->  282 386.20001220703125 (y) ->  236
496
821.7999877929688 (x) ->  282 386.1000061035156 (y) ->  235
567
820.5 (x) ->  282 389.3999938964844 (y) ->  237
638
821.0 (x) ->  282 389.0 (y) ->  237
709
820.5999755859375 (x) ->  282 390.20001220703125 (y) ->  238
780
819.0 (x) ->  281 392.20001220703125 (y) ->  239
851
819.2999877929688 (x) ->  281 390.8999938964844 (y) ->  238
922
817.9000244140625 (x) ->  281 392.70001220703125 (y) ->  239
992
820.7000122070312 (x) ->  282 391.3999938964844 (y) ->  239
1063
821.9000244140625 (x) ->  282 393.70001220703125 (y) ->  240
1134
819.0999755859375 (x) ->  281 391.3999938964844 (y) ->  239
1205
821.0999